In [6]:
import os
import cv2
import shutil
from ultralytics import YOLO

# === RUTAS ===
# Primer modelo: detectar arco superior
model_path_1 = "C:/Users/ana.torres/DetectarArcoSuperior/runs/detect/train/weights/best.pt"
# Segundo modelo: detectar dientes del arco superior
model_path_2 = "C:/Users/ana.torres/BioMod/DetectarDientes/runs/detect/train10/weights/best.pt"

input_folder = "C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/jpeg/"
output_folder = os.path.join(input_folder, "detections")
arco_superior_folder = os.path.join(output_folder, "arco_superior")
labels_folder_1 = os.path.join(output_folder, "labels_arco")
teeth_detection_folder = os.path.join(output_folder, "deteccion_dientes")
labels_folder_2 = os.path.join(output_folder, "labels_dientes")

# === CREAR CARPETAS DE SALIDA ===
os.makedirs(output_folder, exist_ok=True)
os.makedirs(arco_superior_folder, exist_ok=True)
os.makedirs(labels_folder_1, exist_ok=True)
os.makedirs(teeth_detection_folder, exist_ok=True)
os.makedirs(labels_folder_2, exist_ok=True)

# === CARGAR MODELOS ===
model_arco = YOLO(model_path_1)
model_dientes = YOLO(model_path_2)

# === LISTA DE IMÁGENES ===
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# === PROCESO FASE 1: DETECCIÓN DE ARCO SUPERIOR ===
for image_name in image_files:
    image_path = os.path.join(input_folder, image_name)
    results = model_arco(image_path)

    img = cv2.imread(image_path)
    height, width = img.shape[:2]
    save_in_arco_superior = False
    contains_arco_inferior = False
    yolo_annotations = []

    for result in results:
        boxes = result.boxes.xyxy
        confidences = result.boxes.conf
        class_ids = result.boxes.cls.int().tolist()

        for i, (box, conf, class_id) in enumerate(zip(boxes, confidences, class_ids)):
            x1, y1, x2, y2 = map(int, box)
            label = "Arco Superior" if class_id == 0 else "Arco Inferior"
            color = (0, 0, 255) if class_id == 0 else (255, 0, 0)

            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, f"{label} ({conf:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Formato YOLO normalizado
            x_center = ((x1 + x2) / 2) / width
            y_center = ((y1 + y2) / 2) / height
            box_width = (x2 - x1) / width
            box_height = (y2 - y1) / height

            yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

            if class_id == 0:
                save_in_arco_superior = True
            elif class_id == 1:
                contains_arco_inferior = True

    if save_in_arco_superior and not contains_arco_inferior:
        output_path = os.path.join(arco_superior_folder, image_name)
        cv2.imwrite(output_path, img)

        txt_filename = os.path.splitext(image_name)[0] + ".txt"
        txt_path = os.path.join(labels_folder_1, txt_filename)
        with open(txt_path, 'w') as f:
            f.write("\n".join(yolo_annotations))

        print(f"✅ Guardado (Arco): {output_path}")
        print(f"📄 Etiquetas arco: {txt_path}")
    else:
        print(f"❌ No guardada (Arco Inferior detectado o ambas clases presentes): {image_name}")

# === PROCESO FASE 2: DETECCIÓN DE DIENTES EN IMÁGENES FILTRADAS ===
arco_images = [f for f in os.listdir(arco_superior_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

for image_name in arco_images:
    image_path = os.path.join(arco_superior_folder, image_name)
    results = model_dientes(image_path)

    img = cv2.imread(image_path)
    height, width = img.shape[:2]
    yolo_teeth_annotations = []

    for result in results:
        boxes = result.boxes.xyxy
        confidences = result.boxes.conf
        class_ids = result.boxes.cls.int().tolist()

        for i, (box, conf, class_id) in enumerate(zip(boxes, confidences, class_ids)):
            x1, y1, x2, y2 = map(int, box)
            label = f"Diente {class_id}"
            color = (0, 255, 0)

            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, f"{label} ({conf:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # YOLO formato
            x_center = ((x1 + x2) / 2) / width
            y_center = ((y1 + y2) / 2) / height
            box_width = (x2 - x1) / width
            box_height = (y2 - y1) / height

            yolo_teeth_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

    output_teeth_path = os.path.join(teeth_detection_folder, image_name)
    cv2.imwrite(output_teeth_path, img)

    txt_filename = os.path.splitext(image_name)[0] + ".txt"
    txt_path = os.path.join(labels_folder_2, txt_filename)
    with open(txt_path, 'w') as f:
        f.write("\n".join(yolo_teeth_annotations))

    print(f"🦷 Guardado (Dientes): {output_teeth_path}")
    print(f"📄 Etiquetas dientes: {txt_path}")

print("\n🚀 Procesamiento COMPLETO.")



image 1/1 C:\Users\ana.torres\Desktop\python\biomod\DICOM\caso19\jpeg\image0001.jpeg: 640x640 (no detections), 170.6ms
Speed: 5.7ms preprocess, 170.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
❌ No guardada (Arco Inferior detectado o ambas clases presentes): image0001.jpeg

image 1/1 C:\Users\ana.torres\Desktop\python\biomod\DICOM\caso19\jpeg\image0002.jpeg: 640x640 (no detections), 129.1ms
Speed: 5.2ms preprocess, 129.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
❌ No guardada (Arco Inferior detectado o ambas clases presentes): image0002.jpeg

image 1/1 C:\Users\ana.torres\Desktop\python\biomod\DICOM\caso19\jpeg\image0003.jpeg: 640x640 (no detections), 179.6ms
Speed: 5.2ms preprocess, 179.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
❌ No guardada (Arco Inferior detectado o ambas clases presentes): image0003.jpeg

image 1/1 C:\Users\ana.torres\Desktop\python\biomod\DICOM\caso19\jpeg\image0004.jpeg: 640x640 (no dete